In [1]:
# Import libraries
%matplotlib inline
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt

# Read in data
df = pd.read_csv('../data/raw_data/the-office-imdb.csv')

In [2]:
# Rename the columns for simplicity & readability
df.columns=['season', 'title', 'airdate', 'rating', 'votes', 'description', 'directors', 'writers']

# Convert writers column to comma-separated lists
df['writers'] = df['writers'].apply(lambda x : x.replace(' | ', ',').replace(' |', ',').replace(' and ', ',').split(','))

In [3]:
# Custom function to add episode number to IMDB rating dataframe
# Handles special cases/double episodes (below)
# S3 skips episode 12, S4 skips episodes 2/4/6/8, S5 skips episodes 2 and 15, S7 skips episode 12, S9 skips episode 23

s_counter = 1
e_counter = 1
eps = []

for row in df.itertuples():
    if row.season == s_counter:
        if((row.season == 3) and (e_counter == 11)) or ((row.season == 4) and (e_counter in [2,4,6,8])) or ((row.season == 5) and (e_counter in [2,15])) or ((row.season == 7) and (e_counter == 12)) or ((row.season == 9) and (e_counter == 23)):
            e_counter += 1
            eps.append(e_counter)
            e_counter += 1
        else:
            eps.append(e_counter)
            e_counter += 1
    else:
        eps.append(1)
        s_counter += 1
        e_counter = 2
len(eps)

df['episode'] = eps

# Combine multipart episode into one and drop the duplicate
df.loc[(df['title'] == "Niagara: Part 1"), 'description'] = df.iloc[94]['description'] + ' ' + df.iloc[95]['description']
df.loc[(df['title'] == "Niagara: Part 1"), 'title'] = 'Niagara'
df.drop(95, inplace=True)

# Reset indexes
df = df.reset_index(drop=True)

In [4]:
df

,season,title,airdate,rating,votes,description,directors,writers,episode
0,1,Pilot,2005-03-24,7.5,4349,The premiere episode introduces the boss and s...,Ken Kwapis,"[Ricky Gervais, Stephen Merchant, Greg Daniels]",1
1,1,Diversity Day,2005-03-29,8.3,4213,Michael's off color remark puts a sensitivity ...,Ken Kwapis,[B. J. Novak],2
2,1,Health Care,2005-04-05,7.8,3536,Michael leaves Dwight in charge of picking the...,Ken Whittingham,[Paul Lieberstein],3
3,1,The Alliance,2005-04-12,8.1,3428,"Just for a laugh, Jim agrees to an alliance wi...",Bryan Gordon,[Michael Schur],4
4,1,Basketball,2005-04-19,8.4,3745,Michael and his staff challenge the warehouse ...,Greg Daniels,[Greg Daniels],5
...,...,...,...,...,...,...,...,...,...
182,9,Stairmageddon,2013-04-11,8.0,1743,Dwight shoots Stanley with a bull tranquilizer...,Matt Sohn,[Dan Sterling],19
183,9,Paper Airplane,2013-04-25,8.0,1748,The employees hold a paper airplane competitio...,Jesse Peretz,"[Halsted Sullivan, Warren Lieberstein]",20
184,9,Livin' the Dream,2013-05-02,9.0,2439,Dwight becomes regional manager after Andy qui...,Jeffrey Blitz,[Niki Schwartz-Wright],21
185,9,A.A.R.M.,2013-05-09,9.5,3401,Dwight prepares for a marriage proposal and hi...,David Rogers,[Brent Forrester],22


In [ ]:
# Only include columns I am interested in 
rating_df = df[['season', 'episode', 'rating', 'title', 'description', 'airdate']]

# Export to CSV
rating_df.to_csv("../data/ratings-per-ep2.csv")